In [11]:
# !pip install "chonkie[semantic]"
# !pip install markitdown
# !pip install qdrant-client
# !pip install fastembed

In [10]:
# from crewai import Agent, Crew, Process, Task
from crewai import Agent, Crew, Task, LLM, Process
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool
from crewai_tools import PDFSearchTool
# from tools.custom_tool import DocumentSearchTool
from tools.custom_tool import DocumentSearchTool


In [11]:
llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

In [3]:
pdf_tool = DocumentSearchTool(file_path='D:/agentic_rag/knowledge/dspy.pdf')

model.safetensors:   0%|          | 0.00/30.2M [00:00<?, ?B/s]

d:\my_env\rag\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shahzeb\.cache\huggingface\hub\models--minishlab--potion-base-8M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/271k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/684k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

d:\my_env\rag\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shahzeb\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

2025-01-11 23:19:34.753 | ERROR    | fastembed.common.model_management:download_model:274 - Could not download model from HuggingFace: [WinError 1314] A required privilege is not held by the client: '..\\..\\blobs\\a8b3208c2884c4efb86e49300fdd3dc877220cdf' -> 'C:\\Users\\Shahzeb\\AppData\\Local\\Temp\\fastembed_cache\\models--Qdrant--bge-small-en\\snapshots\\8791246cc2a79c7949a4dc0d4a018cbd7d024879\\special_tokens_map.json' Falling back to other sources.
100%|██████████| 77.7M/77.7M [00:24<00:00, 3.13MiB/s]


In [4]:
web_search_tool = SerperDevTool()

In [5]:
import yaml
with open('config/agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [12]:
retriever_agent = Agent(
    config=agents_config['retriever_agent'],
    verbose=True,
    tools=[
        pdf_tool,
        web_search_tool
    ]
)

response_synthesizer_agent = Agent(
    config=agents_config['response_synthesizer_agent'],
    verbose=True
)

retrieval_task = Task(
    description=tasks_config['retrieval_task']['description'],
    expected_output=tasks_config['retrieval_task']['expected_output'],  
    agent=retriever_agent
)

response_task = Task(
    description=tasks_config['response_task']['description'],
    expected_output=tasks_config['response_task']['expected_output'], 
    agent=response_synthesizer_agent
)

In [13]:
crew = Crew(
			agents=[retriever_agent, response_synthesizer_agent], 
			tasks=[retrieval_task, response_task],
			process=Process.sequential,
			verbose=True,
			# process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
		)

2025-01-11 23:29:41,021 - 5952 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [14]:
result = crew.kickoff(inputs={"query": "Who is elon musk?"})
print(result)

2025-01-11 23:29:46,385 - 5952 - llm.py-llm:187 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
2025-01-11 23:29:46,410 - 5952 - llm.py-llm:187 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
2025-01-11 23:29:46,446 - 5952 - llm.py-llm:187 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


# Agent: Retrieve relevant information to answer the user query: Who is elon musk?
## Task: Retrieve the most relevant information from the available sources for the user query: Who is elon musk?



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Retrieve relevant information to answer the user query: Who is elon musk?
## Task: Retrieve the most relevant information from the available sources for the user query: Who is elon musk?



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Retrieve relevant information to answer the user query: Who is elon musk?
## Task: Retrieve the most relevant information from the available sources for the user query: Who is elon musk?



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable